In [ ]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import cv2
import os
import glob
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_data(x): # flag 1 for training data, 0 for testing data
    drct = r"../data/TrainData/"
    
    imgL = []
    label = []
    
    if x:
        df = pd.read_csv('../data/TrainAnnotations.csv')
        for i in range(len(df.annotation)):
            #print(df.file_name[i])
            name = str(df.file_name[i])
            path = drct+name
            img = cv2.imread(path)
            img = cv2.resize(img,(224,224))  
            
            image = img #[:,:,1]
            imgL.append(image) #hog_image_rescaled
            
            if df.annotation[i] == 0: label.append(0)
            elif df.annotation[i] == 1: label.append(1)
            elif df.annotation[i] == 2: label.append(2)
            elif df.annotation[i] == 3: label.append(3)
            elif df.annotation[i] == 4: label.append(4)
           
        data = np.array(imgL) 
        
        clas = np.array(label)
        return data, clas # return five respective image lists

    else: 
        img_dir = "/content/drive/My Drive/C2/data/TestData" # Enter Directory of all images 
        data_path = os.path.join(img_dir,'*g')
        files = glob.glob(data_path)
        data=[]
        data_list = []
        for f1 in files:
          data_list.append(f1)
        data_list = np.array(data_list)
        data_list = np.unique(data_list)

        for img_path in data_list:
          img = cv2.imread(img_path)          
          img = cv2.resize(img,(224,224))     #[#100:300,100:300]           
          image = img    
          data.append(image)#[:,:,1]
          
        data = np.array(data)
        return data

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
x_train, y_train =  load_data(1)

In [ ]:
def get_dir(x,y):
    d_dir = {0:[],1:[],2:[],3:[],4:[]}

    for i in range(len(y)):
        for c in range(5):
            if y[i] == c: 
                d_dir[c].append(x[i])
                break
    return d_dir

In [ ]:
def get_data(dictionary):   # if 1 train if 0 validation
    train_data = dictionary[0] + dictionary[1] + dictionary[2] + dictionary[3] + dictionary[4]
    
    train_label = np.zeros(len(dictionary[0]))
    train_label = np.append(train_label, np.ones(len(dictionary[1])))
    train_label = np.append(train_label, 2*np.ones(len(dictionary[2])))
    train_label = np.append(train_label, 3*np.ones(len(dictionary[3])))
    train_label = np.append(train_label, 4*np.ones(len(dictionary[4])))
    
    data = np.ndarray((len(train_data),224,224,3))
    i = 0
    for img in train_data:
        np.resize(img,(224,224))
        data[i] = img
        i+=1
        
    return data, train_label

In [ ]:
imgL0=[]
imgL1=[]
imgL2=[]
imgL3=[]
imgL4=[]

cla0=[]
cla1=[]
cla2=[]
cla3=[]
cla4=[]

val_d0=[]
val_c0=[]
val_d1=[]
val_c1=[]
val_d2=[]
val_c2=[]
val_d3=[]
val_c3=[]
val_d4=[]
val_c4=[]

for i in range (0,len(y_train)):
  if y_train[i]==0:
    L_0=x_train[i]
    imgL0.append(L_0)
    C_0=y_train[i]
    cla0.append(C_0)

  elif y_train[i]==1:
     L_1=x_train[i]
     imgL1.append(L_1)
     C_1=y_train[i]
     cla1.append(C_1)

  elif y_train[i]==2:
     L_2=x_train[i]
     imgL2.append(L_2)
     C_2=y_train[i]
     cla2.append(C_2)

  elif y_train[i]==3:
    L_3=x_train[i]
    imgL3.append(L_3)
    C_3=y_train[i]
    cla3.append(C_3)

  elif y_train[i]==4:
    L_4=x_train[i]
    imgL4.append(L_4)
    C_4=y_train[i]
    cla4.append(C_4)

train_X0, test_X0, train_y0, test_y0 = train_test_split(imgL0,cla0,test_size=0.08)
train_X1, test_X1, train_y1, test_y1 = train_test_split(imgL1,cla1,test_size=0.12)
train_X2, test_X2, train_y2, test_y2 = train_test_split(imgL2,cla2,test_size=0.31)
train_X3, test_X3, train_y3, test_y3 = train_test_split(imgL3,cla3,test_size=0.30)
train_X4, test_X4, train_y4, test_y4 = train_test_split(imgL4,cla4,test_size=0.20)

train_X0=np.array(train_X0)
vald0=np.array(test_X0)
train_y0=np.array(train_y0)
valc0=np.array(test_y0)

train_X1=np.array(train_X1)
vald1=np.array(test_X1)
train_y1=np.array(train_y1)
valc1=np.array(test_y1)

train_X2=np.array(train_X2)
vald2=np.array(test_X2)
train_y2=np.array(train_y2)
valc2=np.array(test_y2)

train_X3=np.array(train_X3)
vald3=np.array(test_X3)
train_y3=np.array(train_y3)
valc3=np.array(test_y3)

train_X4=np.array(train_X4)
vald4=np.array(test_X4)
train_y4=np.array(train_y4)
valc4=np.array(test_y4)

train_data=np.concatenate((train_X0,train_X1,train_X1[0:160],train_X2,train_X2,train_X2,train_X2,train_X3,train_X3,train_X3,train_X3,train_X4,train_X4))
train_label=np.concatenate((train_y0,train_y1,train_y1[0:160],train_y2,train_y2,train_y2,train_y2,train_y3,train_y3,train_y3,train_y3,train_y4,train_y4))
val_data=np.concatenate((vald0,vald1,vald2,vald3,vald4))
val_label=np.concatenate((valc0,valc1,valc2,valc3,valc4))

print(train_data.shape, train_label.shape, val_data.shape, val_label.shape)

(1931, 224, 224, 3) (1931,) (201, 224, 224, 3) (201,)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_label))
val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_label))
print(train_dataset)

<TensorSliceDataset shapes: ((224, 224, 3), ()), types: (tf.uint8, tf.int64)>


In [ ]:
IMG_SIZE = 224 # All images will be resized to 160x160

def format_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/255) 
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

In [ ]:
train = train_dataset.map(format_example)
validation = val_dataset.map(format_example)
print(type(train))

<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>


In [ ]:
BATCH_SIZE = 10
IMG_SHAPE = (IMG_SIZE,IMG_SIZE,3)

train_batches = train.batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)


In [ ]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

TensorShape([10, 224, 224, 3])

In [ ]:
base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(10, 7, 7, 512)


In [ ]:
base_model.trainable = True

In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 10

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
p_layer = tf.keras.layers.Dense(512,activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.01,l2=0.01))
p1_layer = tf.keras.layers.Dense(512,activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.01,l2=0.01))
p2_layer = tf.keras.layers.Dense(512,activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.01,l2=0.01))
p3_layer = tf.keras.layers.Dense(512,activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.01,l2=0.01))
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)

In [ ]:
prediction_layer = tf.keras.layers.Dense(5,kernel_regularizer=regularizers.l1_l2(l1=0.01,l2=0.01))
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape) 

(10, 5)


In [ ]:
model = tf.keras.Sequential([
  base_model,p1_layer,Dropout(0.6), BatchNormalization(momentum=0.9,epsilon=0.01),
  p2_layer,Dropout(0.6), BatchNormalization(momentum=0.9,epsilon=0.01),
  p3_layer,Dropout(0.6), BatchNormalization(momentum=0.9,epsilon=0.01),
  global_average_layer,Dropout(0.6),BatchNormalization(momentum=0.9,epsilon=0.01),
  p_layer,Dropout(0.5),BatchNormalization(momentum=0.9,epsilon=0.01),
  Dropout(0.5),BatchNormalization(momentum=0.9,epsilon=0.01),prediction_layer
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy','mse'])

In [ ]:
#model.summary()

In [ ]:
# initialize an our data augmenter as an "empty" image data generator
aug = ImageDataGenerator()
# check to see if we are applying "on the fly" data augmentation, and
# if so, re-instantiate the object

aug = ImageDataGenerator(
		rotation_range=15,
		zoom_range=0.15,
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest",
    featurewise_center=True,
    featurewise_std_normalization=True
    )

In [ ]:
len(model.trainable_variables)

34

In [ ]:
initial_epochs =10
validation_steps=5

loss_t,accuracy_t,rmse_t=model.evaluate(train_batches,steps=validation_steps) 
loss0,accuracy0,rmse0 = model.evaluate(validation_batches, steps = validation_steps)

5/5 [==============================] - 0s 24ms/step - loss: 424.9797 - accuracy: 0.0000e+00 - mse: 0.2093


In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 424.98
initial accuracy: 0.00


In [ ]:
w_arr = compute_class_weight('balanced', np.unique(train_label), train_label)
mwrr=(np.max(w_arr))
class_weights = {0:(w_arr[0])/mwrr, 1:(w_arr[1])/mwrr, 2:(w_arr[2])/mwrr, 3:(w_arr[3])/mwrr, 4:(w_arr[4])/mwrr}
class_weights 

{0: 0.7008928571428571,
 1: 0.6993318485523384,
 2: 0.8820224719101124,
 3: 0.8626373626373626,
 4: 1.0}

In [ ]:
#aug.fit(train_data)
#aug_val.fit(val_data)
H = model.fit(aug.flow(train_data,train_label,batch_size=30),
	validation_data=(val_data, val_label), steps_per_epoch=107, epochs=20,class_weight=class_weights)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/100
107/107 [==============================] - 31s 287ms/step - loss: 163.2383 - accuracy: 0.2817 - mse: 6.1021 - val_loss: 69.2285 - val_accuracy: 0.2587 - val_mse: 7.9970
Epoch 2/100
107/107 [==============================] - 30s 279ms/step - loss: 62.3105 - accuracy: 0.4417 - mse: 6.2264 - val_loss: 58.3021 - val_accuracy: 0.1990 - val_mse: 24.1463
Epoch 3/100
107/107 [==============================] - 30s 282ms/step - loss: 68.7747 - accuracy: 0.4917 - mse: 6.3882 - val_loss: 71.1115 - val_accuracy: 0.5572 - val_mse: 7.0598
Epoch 4/100
107/107 [==============================] - 30s 284ms/step - loss: 69.5067 - accuracy: 0.5117 - mse: 6.5697 - val_loss: 74.5564 - val_accuracy: 0.1990 - val_mse: 22.3844
Epoch 5/100
107/107 [==============================] - 30s 278ms/step - loss: 63.8259 - accuracy: 0.5113 - mse: 6.5213 - val_loss: 64.7237 - val_accuracy: 0.1990 - val_mse: 29.0691
Epoch 6/100
107/107 [==============================] - 30s 282ms/step - loss: 57.1311 - accuracy

KeyboardInterrupt: ignored

In [ ]:
acc = H.history['accuracy']
val_acc = H.history['val_accuracy']

loss = H.history['loss']
val_loss = H.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
test_data = load_data(0)


In [ ]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions_test = probability_model.predict(test_data)

In [ ]:
predictions_test[0]

In [ ]:
import csv
pred = []
p0 = 0
p1 = 0
p2 = 0
p3 = 0
p4 = 0
for i in range(len(test_data)):
    if np.argmax(predictions_test[i]) == 0: 
        p0 += 1
        pred.append([1,0,0,0,0])
    elif np.argmax(predictions_test[i]) == 1: 
        p1 += 1
        pred.append([0,1,0,0,0])
    elif np.argmax(predictions_test[i]) == 2: 
        p2 += 1
        pred.append([0,0,1,0,0])
    elif np.argmax(predictions_test[i]) == 3: 
        p3 += 1
        pred.append([0,0,0,1,0])
    elif np.argmax(predictions_test[i]) == 4: 
        p4 += 1
        pred.append([0,0,0,0,1])

df = pd.DataFrame(pred)
df.to_csv("/content/drive/My Drive/C2/data/predict.csv", index=False, header = False)
with open("/content/drive/My Drive/C2/data/predict.csv", "w") as file:
        writer = csv.writer(file)
        for i in range(len(test_data)):
            
            writer.writerow(pred[i])
print(p0,p1,p2,p3,p4)

In [ ]:
y_pred = np.zeros(len(val_label), dtype=np.int8)
val_pred = probability_model.predict(val_data)

for n in range(len(val_label)):
  y_pred[n] = np.argmax(val_pred[n])
  #print(y_pred[n],val_label[n])

val = []
v0 = 0
v1 = 0
v2 = 0
v3 = 0
v4 = 0
for i in range(len(val_label)):
    if val_label[i] == 0: 
        v0 += 1
        val.append([1,0,0,0,0])
    elif val_label[i] == 1: 
        v1 += 1
        val.append([0,1,0,0,0])
    elif val_label[i] == 2: 
        v2 += 1
        val.append([0,0,1,0,0])
    elif val_label[i] == 3: 
        v3 += 1
        val.append([0,0,0,1,0])
    elif val_label[i] == 4: 
        v4 += 1
        val.append([0,0,0,0,1])

val = np.array(val)
#print(val_pred)
#print(val)

In [ ]:
target_names = [0,1,2,3,4]
plt.figure(1)
c_matrix= confusion_matrix(val_label, y_pred,labels=[0,1,2,3,4,])

fig, ax = plt.subplots()
im = ax.imshow(np.array(c_matrix))
               
ax.set_xticks(np.arange(len(target_names)))
ax.set_yticks(np.arange(len(target_names)))    

ax.set_xticklabels(target_names)
ax.set_yticklabels(target_names)

ax.margins(1., y = 1)
plt.setp(ax.get_xticklabels(), rotation=0, ha="right",rotation_mode="anchor")
for i in range(len(target_names)):
    for j in range(len(target_names)):
        text = ax.text(j, i, c_matrix[i, j],
                       ha='center', verticalalignment='center', color='w')
fig.tight_layout(pad = 3)
plt.show()

In [ ]:

print(classification_report(val_label, y_pred, target_names = ['class 0', 'class 1', 'class 2', 'class 3', 'class 4']))
